# RLlib Sample Application: Taxi-v3

This example uses [RLlib](https://ray.readthedocs.io/en/latest/rllib.html) to train a policy with the `Taxi-v3` environment ([gym.openai.com/envs/Taxi-v3/](https://gym.openai.com/envs/Taxi-v3/)).

For more background about this problem, see:

* ["Hierarchical Reinforcement Learning with the MAXQ Value Function Decomposition"](https://arxiv.org/abs/cs/9905014), [Thomas G. Dietteric](https://twitter.com/tdietterich)
* ["Reinforcement Learning: let’s teach a taxi-cab how to drive"](https://towardsdatascience.com/reinforcement-learning-lets-teach-a-taxi-cab-how-to-drive-4fd1a0d00529), [Valentina Alto](https://twitter.com/AltoValentina)

In [1]:
import ray
import ray.rllib.agents.ppo as ppo

In [2]:
ray.init(ignore_reinit_error=True)

2020-05-01 12:29:07,344	INFO resource_spec.py:212 -- Starting Ray with 4.1 GiB memory available for workers and up to 2.06 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-05-01 12:29:07,683	INFO services.py:1148 -- View the Ray dashboard at localhost:8267


{'node_ip_address': '192.168.1.149',
 'redis_address': '192.168.1.149:22890',
 'object_store_address': '/tmp/ray/session_2020-05-01_12-29-07_333908_27203/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-05-01_12-29-07_333908_27203/sockets/raylet',
 'webui_url': 'localhost:8267',
 'session_dir': '/tmp/ray/session_2020-05-01_12-29-07_333908_27203'}

In [3]:
print(f'Dashboard URL: http://{ray.get_webui_url()}')

Dashboard URL: http://localhost:8267


Next we'll train an RLlib policy with the `Taxi-v3` environment.

By default, training runs for `10` iterations. Increase the `N_ITER` setting if you want to see the resulting rewards improve.
Also note that *checkpoints* get saved after each iteration into the `/tmp/ppo/taxi` directory.

> **Note:** If you prefer to use a different directory root than `/tmp`, change it in the next cell **and** in the `rllib rollout` command below.

In [4]:
SELECT_ENV = "Taxi-v3"
N_ITER = 10

config = ppo.DEFAULT_CONFIG.copy()
config["log_level"] = "WARN"

reward_history = []

agent = ppo.PPOTrainer(config, env=SELECT_ENV)

for _ in range(N_ITER):
    result = agent.train()
    print(result)

    max_reward = result["episode_reward_max"]
    reward_history.append(max_reward)

    file_name = agent.save("/tmp/ppo/taxi")
    print(f'\ncheckpoint saved to {file_name}')

2020-05-01 12:31:37,269	INFO trainer.py:428 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2020-05-01 12:31:37,301	INFO trainer.py:585 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
/Users/deanwampler/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
2020-05-01 12:31:40,012	INFO trainable.py:217 -- Getting current IP.
2020-05-01 12:31:40,012	WARNING util.py:37 -- Install gputil for GPU system monitoring.


(pid=29843) /Users/deanwampler/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=29843)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=29847) /Users/deanwampler/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=29847)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
{'episode_reward_max': -613.0, 'episode_reward_min': -827.0, 'episode_reward_mean': -741.6, 'episode_len_mean': 199.65, 'episodes_this_iter': 20, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [-740.0, -827.0, -794.0, -767.0, -704.0, -613.0, -803.0, -677.0, -722.0, -695.0, -740.0, -695.0, -758.0, -722.0, -758.0, -767.0, -767.0, -758.0, -776.0, -749.0], 'episode_lengths': [200, 200, 200, 200, 200, 19

In [5]:
print(reward_history)

[-613.0, -513.0, -236.0, -236.0, -139.0, -114.0, -77.0, -77.0, -30.0, -30.0]


Do the episode rewards increase after multiple iterations?
That shows whether the policy is improving.

Also, print out the policy and model to see the results of training in detail…

In [6]:
import pprint

policy = agent.get_policy()
model = policy.model

pprint.pprint(model.variables())
pprint.pprint(model.value_function())

print(model.base_model.summary())

[<tf.Variable 'default_policy/fc_1/kernel:0' shape=(500, 256) dtype=float32>,
 <tf.Variable 'default_policy/fc_1/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'default_policy/fc_value_1/kernel:0' shape=(500, 256) dtype=float32>,
 <tf.Variable 'default_policy/fc_value_1/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'default_policy/fc_2/kernel:0' shape=(256, 256) dtype=float32>,
 <tf.Variable 'default_policy/fc_2/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'default_policy/fc_value_2/kernel:0' shape=(256, 256) dtype=float32>,
 <tf.Variable 'default_policy/fc_value_2/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'default_policy/fc_out/kernel:0' shape=(256, 6) dtype=float32>,
 <tf.Variable 'default_policy/fc_out/bias:0' shape=(6,) dtype=float32>,
 <tf.Variable 'default_policy/value_out/kernel:0' shape=(256, 1) dtype=float32>,
 <tf.Variable 'default_policy/value_out/bias:0' shape=(1,) dtype=float32>]
<tf.Tensor 'Reshape:0' shape=(?,) dtype=float32>
Model: "model"
______

Next we'll use the [`rollout` script](https://ray.readthedocs.io/en/latest/rllib-training.html#evaluating-trained-policies) to evaluate the trained policy.

This visualizes the "taxi" agent operating within the simulation: picking up fares, driving, turning, doing dropoffs, etc.

In [7]:
! rllib rollout \
    /tmp/ppo/taxi/checkpoint_2/checkpoint-2 \
    --config "{\"env\": \"Taxi-v3\"}" --run PPO \
    --steps 2000

2020-05-01 12:34:28,084	INFO resource_spec.py:212 -- Starting Ray with 4.3 GiB memory available for workers and up to 2.16 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-05-01 12:34:28,418	INFO services.py:1148 -- View the Ray dashboard at localhost:8268
2020-05-01 12:34:29,083	INFO trainer.py:428 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2020-05-01 12:34:29,120	INFO trainer.py:585 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
/Users/deanwampler/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
2020-05-01 12:34:32,022	INFO trainable.py:217 -- Getting current IP.
2020-05-01 12:34:32,022	WARNING util.py:37 -- Install gputil for GPU system monitoring.
2020

The rollout uses the second saved checkpoint, evaluated through `2000` steps.
Modify the path to view other checkpoints.

---

Finally, launch [TensorBoard](https://ray.readthedocs.io/en/latest/rllib-training.html#getting-started) then follow the instructions (copy/paste the URL it generates) to visualize key metrics from training with RLlib…

In [ ]:
!tensorboard --logdir=~/ray_results/

Next, go through any of other remaining lessons:

* [04a: Application: Mountain Car](04a-Application-Mountain-Car.ipynb) -- Based on the `MountainCar-v0` environment from OpenAI Gym.
* [04c: Application: Frozen Lake](04c-Application-Frozen-Lake.ipynb) -- Based on the `FrozenLake-v0` environment from OpenAI Gym.